In [1]:
# Standard library imports
from io import StringIO
import importlib
import math
import os
import pdb
import re
import regex
import sys 

# Third party imports
import dateparser
import reverse_geocoder as rg
import numpy as np
import pandas as pd

# Local application imports
import mender_tools as mt

# Reading the csv file into a dataframe

In [2]:
df = pd.read_csv('blazes/fires.csv')

# Set options to display all rows and columns in Jupyter notebook

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Checking out the dataframe

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.head()

# Renaming columns names

In [ ]:
df.columns

In [4]:
df.columns = ['Cause', 'Coordinates', 'Date', 'Fuels Involved', 'Incident', 'Incident Type', 'Location', 'Perimeter Contained (%)', 'Personnel Involved', 'Fire Size (Acres)']

# Removing rows with all values missing and striping whitespaces

In [5]:
# Checking total number of rows with all cells empty
df.isnull().all(axis=1).sum()


9

In [6]:
# Drop all the rows with all cells empty
df.dropna(how='all', inplace=True)

In [7]:
# Reseting the index of the dataframe
df.reset_index(drop = True, inplace = True)

In [8]:
# Remove trailing and leading whitespaces
df.loc[:,:] = df.applymap(lambda x: x.strip() if type(x)==str else x)

In [9]:
# Locate rows of duplicate data
dups = df.duplicated()
print(dups.any())

False


# Making a backup

In [ ]:
''' In order to make a copy of the dataframe we need two different instances of the dataframe, if we don't do that any changes
made to any of the variables that reference to dataframe will modify the other one. In that case we use the method copy().'''
wildfire_df = df.copy()

# Arrange Numeric Columns

## Preparing column 'Fire Size (Acres)'

In [ ]:
# Copy values of acres in column 'Personnel Involved' that pertain to column 'Fire Size (Acres)'
wildfire_df = mt.emend_values(df, wildfire_df, 'Personnel Involved', 'Fire Size (Acres)', r'.*\s*Acres')

In [ ]:
# Remove the string 'Acres' that comes along with the digits
wildfire_df = mt.strip_symbol(wildfire_df, 'Fire Size (Acres)', 'Acres')

In [ ]:
# Convert the string digits to integer values
wildfire_df = mt.convert_to_int(wildfire_df, 'Fire Size (Acres)')

In [ ]:
 # Check for value types 
wildfire_df['Fire Size (Acres)'].apply(type)

## Preparing column 'Perimeter Contained (%)'

In [ ]:
# Get values in column 'Fire Size (Acres)' that belong to column 'Perimeter Contained (%)'
wildfire_df = mt.emend_values(df, wildfire_df, 'Fire Size (Acres)', 'Perimeter Contained (%)', r'^\d*[.]{0,1}\d*\s*%$')

In [ ]:
# Get rid of '%' symbol
wildfire_df = mt.strip_symbol(wildfire_df, 'Perimeter Contained (%)', '%')

In [ ]:
# Convert float values to integer
wildfire_df = mt.convert_to_int(wildfire_df, 'Perimeter Contained (%)')

In [ ]:
# Total of value types
wildfire_df['Perimeter Contained (%)'].apply(type).value_counts()

In [ ]:
# Convert string values to NaN
wildfire_df = mt.str_to_nan(wildfire_df, 'Perimeter Contained (%)')

In [ ]:
# Total of value types
wildfire_df['Perimeter Contained (%)'].apply(type).value_counts()

In [ ]:
# Total of null values
wildfire_df['Perimeter Contained (%)'].isnull().sum()

## Preparing column 'Personnel Involved'

In [ ]:
# Check elements type
wildfire_df['Personnel Involved'].apply(type)

In [ ]:
# Get total value types
wildfire_df['Personnel Involved'].apply(type).value_counts()

In [ ]:
# Get total sum of null values
wildfire_df['Personnel Involved'].isnull().sum()

In [ ]:
# Convert string digits to int
wildfire_df = mt.convert_to_int(wildfire_df, 'Personnel Involved')

In [ ]:
# Convert no digits strings to NANs
wildfire_df = mt.str_to_nan(wildfire_df, 'Personnel Involved')

In [ ]:
wildfire_df = mt.convert_to_string(wildfire_df, 'Personnel Involved')

In [ ]:
# Set column to 'Int64' to have a column both with integer and NAN values
wildfire_df['Personnel Involved'] = wildfire_df['Personnel Involved'] .astype('Int64')

In [ ]:
# Show value type for shown columns side by side
wildfire_df[['Fire Size (Acres)','Personnel Involved','Perimeter Contained (%)']].applymap(type)

In [ ]:
# To finalize the numeric columns preparation we save the dataframe as a csv file
wildfire_df.to_csv('blazes/fires_v2.csv', index = False)

# Arrange Categorical Columns

## Preparing Column 'Coordinates'

In [10]:
# Read the csv file into a new dataframe
wildf = pd.read_csv('blazes/fires_v2.csv')

In [11]:
# Make a copy of the dataframe
wildf_dfv2 = wildf.copy()

In [12]:
# Copy the values from column 'Location' to the corresponding cells in column 'Coordinates'
wildf_dfv2 = mt.emend_values(df, wildf_dfv2, 'Location', 'Coordinates', r'-?\d+\.?\d+\s*latitude,?\s*-?\d+\.?\d+\s*longitude')

In [13]:
# Copy the values from column 'Cause' to the corresponding cells in column 'Coordinates'
wildf_dfv2 = mt.emend_values(df, wildf_dfv2, 'Cause', 'Coordinates', r'-?\d+\.?\d+\s*latitude,?\s*-?\d+\.?\d+\s*longitude')

In [14]:
wildf_dfv2['Coordinates'].isnull().sum()

11

In [ ]:
# Create a new column 'Latitude' with the values that go along with 'latitude' in column 'Coordinates'
wildf_dfv2 = mt.create_new_col(wildf_dfv2, r'-?\d+\.?\d+\s*(?=latitude)', 'Coordinates', 'Latitude')

In [ ]:
# Create a new column 'Longitude' with the values that go along with 'longitude' in column 'Coordinates'
wildf_dfv2 = mt.create_new_col(wildf_dfv2, r'-?\d+\.?\d+\s*(?=longitude)', 'Coordinates', 'Longitude')

In [ ]:
# Show value types totals
wildf_dfv2['Latitude'].apply(type).value_counts()

In [ ]:
# Show empty strings totals
(wildf_dfv2['Latitude'] == '').sum()

In [ ]:
# Fill empty strings with NaNs values in column 'Longitude'
wildf_dfv2['Longitude'].replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [ ]:
# Fill empty strings with NaNs values in column 'Latitude'
wildf_dfv2['Latitude'].replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [ ]:
# Show value types totals
wildf_dfv2['Latitude'].apply(type).value_counts()

In [ ]:
# Show null values totals
wildf_dfv2['Latitude'].isnull().sum()

In [ ]:
# Convert strings to float in column 'Latitude'
wildf_dfv2['Latitude'] = pd.to_numeric(wildf_dfv2['Latitude'], downcast='float')

In [ ]:
# Convert strings to float in column 'Longitude'
wildf_dfv2['Longitude'] = pd.to_numeric(wildf_dfv2['Longitude'], downcast='float')

In [ ]:
# Drop column 'Coordinates'
wildf_dfv2.drop(['Coordinates'], axis=1, inplace=True)

## Preparing Column 'Date'

In [ ]:
# Copy the values from column 'Cause' to the corresponding cells in column 'Date'
wildf_dfv2 = mt.emend_values(df, wildf_dfv2, 'Cause', 'Date', r'(\d{2}:\d{2}\s?(AM|PM))$')

In [ ]:
# Remove string 'approx.' from the string containing the day and hour
wildf_dfv2 = mt.strip_symbol(wildf_dfv2, 'Date', 'approx.')

In [ ]:
# Convert date entries from 'Date' column to ISO format
wildf_dfv2 = mt.convert_to_isodate(wildf_dfv2, 'Date')

In [ ]:
# Show columns 'Date' and 'ISO Date' side by side
wildf_dfv2.loc[:,['Date', 'ISO Date']].head()

In [ ]:
# Drop column 'Date'
wildf_dfv2.drop(['Date'], axis=1, inplace=True)

## Preparing Column 'Cause'

In [ ]:
# Get rid of strings of dates
wildf_dfv2 = mt.strip_string(wildf_dfv2, 'Cause', r'-?\d+\.?\d+\s*latitude,?\s*-?\d+\.?\d+\s*longitude')

In [ ]:
# Get rid of strings of coordinates
wildf_dfv2 = mt.strip_string(wildf_dfv2, 'Cause', r'(\d{2}:\d{2}\s?(AM|PM))$')

In [ ]:
# Remove trailing and leading whitespaces
wildf_dfv2.loc[:, 'Cause'] = wildf_dfv2.loc[:, 'Cause'].apply(lambda x: x.strip() if type(x)==str else x)

In [ ]:
# Replace empty strings with NaN values
wildf_dfv2['Cause'].replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [ ]:
# Create and populate set with causes of fire
causes = set()
for item in wildf_dfv2['Cause']:
    if isinstance(item, str):
        if item not in causes:
            causes.add(item)
print(causes)

In [ ]:
# List causes of fire from set 'causes'
i=1
for item in causes:
    print(i,'->',item)
    i += 1

In [ ]:
# Ad hoc function for replacing and fixing redundant values in column 'Cause'
result = ""
def is_match(pattern, x, word):
    global result
    if isinstance(x, str):
        match = re.search(pattern, x)
        if match:
            result = x.replace(match.string, word)
        else:
            return False
    else:
        return False
    return True    

In [ ]:
# Using function 'is_match' with a lambda function
wildf_dfv2.loc[:, 'Cause'] = wildf_dfv2.loc\
[:, 'Cause'].apply(lambda x:x.replace(x, result) if\
is_match(r'Lightning', x, 'Lightning') else x)

In [ ]:
# Using function 'is_match' with a lambda function
wildf_dfv2.loc[:, 'Cause'] = wildf_dfv2.loc\
[:, 'Cause'].apply(lambda x:x.replace(x, result)\
if is_match(r'Human', x, 'Human Caused') else x)

In [ ]:
# Using function 'is_match' with a lambda function
wildf_dfv2.loc[:, 'Cause'] = wildf_dfv2.loc\
[:, 'Cause'].apply(lambda x:x.replace(x, result)\
                   if is_match(r'Unk[n]?own', x, 'Unknown') else x)

In [ ]:
wildf_dfv2.loc[:, 'Cause'].head()

## Creating column Region

In [ ]:
# Get the location given by latitude and longitude coordinates
def get_location(coordinates):
    return rg.search(coordinates)

In [ ]:
# Function for creating column 'Region' based on coordinates data
def create_region_col(df):
    i = 0
    for latitude, longitude in zip(wildf_dfv2['Latitude'], wildf_dfv2['Longitude']):        
        if (not math.isnan(latitude) and not math.isnan(longitude)):
           coordinates = (latitude, longitude)
           location = get_location(coordinates)
           df.at[i, 'Region'] = list(location[0].values())[3]
           i += 1            
        else:
           i += 1

In [ ]:
# Call to function create_region_col
create_region_col(wildf_dfv2)

In [ ]:
# Checking null values after last call to create_region_col
wildf_dfv2.loc[:,  'Region'].isnull().sum()

In [ ]:
wildf_dfv2.loc[15:24, ['Location', 'Latitude', 'Longitude', 'Region']]

## Appendix Section

In [ ]:
wildf_dfv21 = wildf_dfv2.copy()

In [ ]:
wildf_dfv21['Region'].isnull().sum()

In [ ]:
# Dictionary with US states names and their abbrevations
us_abbrev_dic={'Alabama': 'AL', 'Alaska': 'AK', 'Arizona':'AZ', 'Arkansas':'AR',
               'California': 'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE',
               'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
               'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
               'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
               'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
               'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
               'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
               'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
               'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
               'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
               'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
               'Wisconsin': 'WI', 'Wyoming': 'WY'  
              }
st = us_abbrev_dic

In [ ]:
# Ad hoc function for completing states names in column 'Region' based on column 'Location' data
def loc_to_state(df, col):
    i=0
    flag=0
    list_count = 0
    wordup = ''
    for sentence in df[col]:
        if not isinstance(df.at[i,'Region'], str):
            if isinstance(sentence, str):
                word_list = re.findall(r'\w+', sentence)
                word_list_len = len(word_list)
                for word in word_list:
                    list_count += 1
                    if len(word)>2:
                        wordup = word.upper()
                    else:
                        wordup = word
                    for st_name, st_abbrev in st.items():
                        upper_st = st_name.upper()
                        if (wordup == upper_st or wordup == st_abbrev):
                            df.at[i,'Region'] = st_name
                            i+=1
                            flag = 1
                            break
                    if (list_count == word_list_len and flag ==0):
                        list_count = 0
                        i+=1
                    elif(flag == 1):
                        list_count = 0
                        flag = 0
                        break
            else:
                i+=1
        else:
            i+=1

In [ ]:
# Call to function loc_to_state
loc_to_state(wildf_dfv21, 'Location')

In [ ]:
# Listing the rows of 'Location' that will be googled
i = 0
indexes = list()
for item, name in zip(wildf_dfv21['Region'], wildf_dfv21['Location']):
    if (isinstance(item, float) and not isinstance(name, float)):
        indexes.append(wildf_dfv21.index[wildf_dfv21['Location'] == name].tolist())
        print(indexes[i][0], '->', name)
        i += 1  

In [ ]:
# Dictionary with the last states with keys serving as dataframe corresponding indexes
last_st = {20:'California', 23:'Colorado', 26: 'Arizona', 33: 'Arizona',\
          53: 'Oregon', 56: 'Arizona', 61: 'Idaho', 67: 'Nevada', 76: 'California',\
           78: 'California', 97: 'Arizona', 98: 'California'}

In [ ]:
# Set last missing states
for key, values in last_st.items():
        wildf_dfv21.at[key, 'Region'] = values

In [ ]:
wildf_dfv21.loc[[12,40], ['Incident', 'Location', 'Region']].head(41)

In [ ]:
wildf_dfv21.at[12, 'Region'] = 'New Mexico'
wildf_dfv21.at[40, 'Region'] ='California'   

In [ ]:
# Drop column 'Location'
wildf_dfv21.drop(['Location'], axis=1, inplace=True)

In [ ]:
# Drop column 'Incident'
wildf_dfv21.drop(['Incident'], axis=1, inplace=True)

In [ ]:
# Save dataframe to csv file
wildf_dfv21.to_csv('blazes/fires_v3.csv', index = False)

## Final Tweakings

In [ ]:
# Read the csv file into a new dataframe
wildf = pd.read_csv('blazes/fires_v3.csv')

In [ ]:
# Make a copy
wildf_dfv3 = wildf.copy()

In [ ]:
# Show selected columns where column 'Incident Type' is 'Burned Area Emergency Response'
exclude = ['Latitude','Longitude', 'Fuels Involved', 'Region']
wildf_dfv3.loc[wildf_dfv3['Incident Type'] == 'Burned Area Emergency Response', wildf_dfv3.columns.difference(exclude, sort=False)]

In [ ]:
wildf_dfv3.drop(index=wildf_dfv3[wildf_dfv3['Incident Type'] == 'Burned Area Emergency Response'].index, inplace=True)

In [ ]:
# Checking total number of rows with all cells empty
wildf_dfv3.isnull().all(axis=1).sum()

In [ ]:
# Drop all the rows with all cells empty
wildf_dfv3.dropna(how='all', inplace=True)

In [ ]:
wildf_dfv3.shape

In [ ]:
# Converting float values to integers. When saving dataframe to csv int values are saved as float
# wildf_dfv3[['Perimeter Contained (%)','Personnel Involved','Fire Size (Acres)']] = wildf_dfv3[['Perimeter Contained (%)','Personnel Involved','Fire Size (Acres)']].astype('Int64')
# Or convert only 'Pesonnel involved' and 'Fire Size (Acres)' to integer values'
wildf_dfv3[['Personnel Involved','Fire Size (Acres)']] = wildf_dfv3[['Personnel Involved','Fire Size (Acres)']].astype('Int64')

In [ ]:
# Show final dataset
exclude = ['Latitude','Longitude', 'Fuels Involved']
wildf_dfv3.loc[:, wildf_dfv3.columns.difference(exclude, sort=False)].head(100)

In [ ]:
wildf_dfv3.to_csv('blazes/fires_v4.csv', index = False)

# Addendum

In [ ]:
# Dictionary with U.S states-capitals
us_capital_dic={
    'Alabama': 'Montgomery',
    'Alaska': 'Juneau',
    'Arizona':'Phoenix',
    'Arkansas':'Little Rock',
    'California': 'Sacramento',
    'Colorado':'Denver',
    'Connecticut':'Hartford',
    'Delaware':'Dover',
    'Florida': 'Tallahassee',
    'Georgia': 'Atlanta',
    'Hawaii': 'Honolulu',
    'Idaho': 'Boise',
    'Illinois': 'Springfield',
    'Indiana': 'Indianapolis',
    'Iowa': 'Des Monies',
    'Kansas': 'Topeka',
    'Kentucky': 'Frankfort',
    'Louisiana': 'Baton Rouge',
    'Maine': 'Augusta',
    'Maryland': 'Annapolis',
    'Massachusetts': 'Boston',
    'Michigan': 'Lansing',
    'Minnesota': 'St. Paul',
    'Mississippi': 'Jackson',
    'Missouri': 'Jefferson City',
    'Montana': 'Helena',
    'Nebraska': 'Lincoln',
    'Nevada': 'Carson City',
    'New Hampshire': 'Concord',
    'New Jersey': 'Trenton',
    'New Mexico': 'Santa Fe',
    'New York': 'Albany',
    'North Carolina': 'Raleigh',
    'North Dakota': 'Bismarck',
    'Ohio': 'Columbus',
    'Oklahoma': 'Oklahoma City',
    'Oregon': 'Salem',
    'Pennsylvania': 'Harrisburg',
    'Rhode Island': 'Providence',
    'South Carolina': 'Columbia',
    'South Dakoda': 'Pierre',
    'Tennessee': 'Nashville',
    'Texas': 'Austin',
    'Utah': 'Salt Lake City',
    'Vermont': 'Montpelier',
    'Virginia': 'Richmond',
    'Washington': 'Olympia',
    'West Virginia': 'Charleston',
    'Wisconsin': 'Madison',
    'Wyoming': 'Cheyenne'  
}

In [ ]:
# Slicing tips
wildf_dfv2.loc[wildf_dfv2['Region'].isnull(), ['Location', 'Region']].head(100)